# Classiq code for discrete quantum walk

This notebook shows how to generate data for discrete quantum walk using `classiq`.

In [1]:
import time

from classiq import *

SIZE = 6
MAX_WIDTH = 40
constraints = Constraints(optimization_parameter="cx", max_width=MAX_WIDTH)

In [2]:
# define increment circuit as an MCX cascade
@qfunc
def increment(x: QArray):
    repeat(
        x.len - 1, lambda i: control(x[0 : x.len - 1 - i], lambda: X(x[x.len - 1 - i]))
    )
    X(x[0])


@qfunc
def single_step_walk(
    coin: QBit,  # coin
    x: QNum,  # position
):
    H(coin)
    control(coin == 0, lambda: increment(x), lambda: invert(lambda: increment(x))),

In [3]:
from classiq import CustomHardwareSettings, Preferences

preferences = Preferences(
    custom_hardware_settings=CustomHardwareSettings(basis_gates=["cx", "u"]),
    transpilation_option="custom",
    debug_mode=False,
)

## Example for getting a data point

In [4]:
start_time = time.time()


@qfunc
def main(x: Output[QNum[SIZE, UNSIGNED, 0]]):
    allocate(x)
    coin = QBit()
    allocate(coin)
    single_step_walk(coin, x)


write_qmod(main, "quantum_walk_classiq")
qprog = synthesize(main, constraints=constraints, preferences=preferences)
compilation_time = time.time() - start_time

width = qprog.data.width
depth = qprog.transpiled_circuit.depth
cx_counts = qprog.transpiled_circuit.count_ops["cx"]

print(f"==== classiq for {SIZE}==== time {compilation_time}")

==== classiq for 6==== time 16.50112295150757
